# 飞桨常规赛：遥感影像地块分割8月第1名方案

# 环境准备
本项目采用paddleseg套件实现了遥感影像地块的分割，paddleseg版本号为v2.1版本

In [ ]:
# 下载paddlesegv2.1版本
!git clone -b release/2.1 https://gitee.com/paddlepaddle/PaddleSeg.git

# 数据集准备和分析
首先将数据集解压缩，然后运行work目录下的make_data_list.py划分出训练集和验证集文件列表

In [ ]:
# 解压训练数据集到work目录下
!unzip -oq -d /home/aistudio/work /home/aistudio/data/data80164/train_and_label.zip
# 解压测试数据集到work目录下
!unzip -oq -d /home/aistudio/work /home/aistudio/data/data80164/img_test.zip
# 生成文件列表文件
!python /home/aistudio/work/make_data_list.py

## 分析数据集
数据集的分析参考了参考链接中的分析方法，统计了数据中四种类型的面积分布情况。但实际本方案并没有针对分析后的数据集做特定的优化，后续会继续尝试参考链接中的思路。

In [ ]:
import cv2
import numpy as np

NUM_CLASSES = 4

area = {i : 0 for i in range(NUM_CLASSES)}
area_proportion = {i : {0 : 0, 1 : 0, 2 : 0, 3 : 0} for i in range(NUM_CLASSES)}
area[255] = 0
image_num = 0

def calc(image, num_classes=NUM_CLASSES):
    label_image = np.array(image)
    for cls in range(num_classes):
        area[cls] += np.count_nonzero(label_image == cls)
    area[255] += np.count_nonzero(label_image == 255)

def area_calc(image, num_classes=NUM_CLASSES):
    label_image = np.array(image)
    image_area = label_image.shape[0] * label_image.shape[1]
    for cls in range(num_classes):
        proportion = np.count_nonzero(label_image == cls) / float(image_area)
        if proportion < 0.01:
            area_proportion[cls][0] += 1
        elif proportion < 0.02:
            area_proportion[cls][1] += 1
        elif proportion < 0.8:
            area_proportion[cls][2] += 1
        else:
            area_proportion[cls][3] += 1


# 统计四种类型的面积占比
train_file_dir = '/home/aistudio/work/train_list.txt'
val_file_dir = '/home/aistudio/work/val_list.txt'
with open(train_file_dir, 'r') as f:
    for line in f.readlines():
        label_dir = line.split()[1]
        # print(label_dir)
        image_label = cv2.imread(label_dir, cv2.IMREAD_GRAYSCALE)
        calc(image_label)
        area_calc(image_label)
        image_num += 1
        # break

for cls in range(NUM_CLASSES):
    area[cls] = area[cls] / (image_num * 256.0 * 256.0)
area[255] = area[255] / (image_num * 256.0 * 256.0)
print(area)
print(area_proportion)

## 数据集代码准备
模型训练借助了paddleseg套件，在paddleseg中新加入了一种数据集RemoteSensing。本方案新加入数据集的思路实际上是为后续模型持续优化做准备，目前已实现的方案并没有对数据集做特别的处理，下面简述新加数据集时涉及到的修改padddleseg源码步骤
1. 在work/PaddleSeg/paddleseg/datasets中添加python文件remote_sensing.py
2. 修改work/PaddleSeg/paddleseg/datasets中的__init__.py，添加一句话from .remote_sensing import RemoteSensing

# 模型训练
从数据集分析结果来看，各个类别像素占整个图片的面积比例很小，所以选择了参考链接[1]中的ocrnet+hrnet模型，在训练时分了两个阶段，第一个阶段的loss采用了CrossEntropyLoss，训练了大约30个epoch，第二阶段采用的loss为CrossEntropyLoss + LovaszSoftmaxLoss，训练了大约10个epoch。两阶段训练的思路参考了paddleseg的loss说明https://gitee.com/paddlepaddle/PaddleSeg/blob/release/2.1/docs/module/loss/lovasz_loss.md

在训练前，将make_data_list.py产生的文件列表放在/home/aistudio/work/multi_class_datalist目录下

In [ ]:
# 调用paddleseg进行模型训练前30个epoch
!python /home/aistudio/work/PaddleSeg/train.py \
       --config /home/aistudio/work/PaddleSeg/configs/quick_start/ocrnet_hrnet_256x256_ce.yml \
       --do_eval \
       --use_vdl \
       --save_interval 5000 \
       --save_dir output_ocrnet

In [ ]:
# 调用paddleseg进行模型训练后10个epoch，在前30个epoch的基础上继续训练
!python /home/aistudio/work/PaddleSeg/train.py \
       --config /home/aistudio/work/PaddleSeg/configs/quick_start/ocrnet_hrnet_256x256_ce_and_lovasz.yml \
       --do_eval \
       --use_vdl \
       --save_interval 5000 \
       --resume_model /home/aistudio/output_ocrnet/iter_160000 \
       --save_dir output_ocrnet

# 模型预测
模型预测时希望将预测的结果直接作为提交结果，但是paddleseg默认预测的结果是增加权重后生成的图片，所以对paddleseg的源码进行了修改。修改的文件为work/PaddleSeg/paddleseg/core/predict.py，修改后是下面这段代码

In [ ]:
# 修改predict函数
def predict(model,
            model_path,
            transforms,
            image_list,
            image_dir=None,
            save_dir='output',
            aug_pred=False,
            scales=1.0,
            flip_horizontal=True,
            flip_vertical=False,
            is_slide=False,
            stride=None,
            crop_size=None):
    """
    predict and visualize the image_list.

    Args:
        model (nn.Layer): Used to predict for input image.
        model_path (str): The path of pretrained model.
        transforms (transform.Compose): Preprocess for input image.
        image_list (list): A list of image path to be predicted.
        image_dir (str, optional): The root directory of the images predicted. Default: None.
        save_dir (str, optional): The directory to save the visualized results. Default: 'output'.
        aug_pred (bool, optional): Whether to use mulit-scales and flip augment for predition. Default: False.
        scales (list|float, optional): Scales for augment. It is valid when `aug_pred` is True. Default: 1.0.
        flip_horizontal (bool, optional): Whether to use flip horizontally augment. It is valid when `aug_pred` is True. Default: True.
        flip_vertical (bool, optional): Whether to use flip vertically augment. It is valid when `aug_pred` is True. Default: False.
        is_slide (bool, optional): Whether to predict by sliding window. Default: False.
        stride (tuple|list, optional): The stride of sliding window, the first is width and the second is height.
            It should be provided when `is_slide` is True.
        crop_size (tuple|list, optional):  The crop size of sliding window, the first is width and the second is height.
            It should be provided when `is_slide` is True.

    """
    utils.utils.load_entire_model(model, model_path)
    model.eval()
    nranks = paddle.distributed.get_world_size()
    local_rank = paddle.distributed.get_rank()
    if nranks > 1:
        img_lists = partition_list(image_list, nranks)
    else:
        img_lists = [image_list]

    added_saved_dir = os.path.join(save_dir, 'added_prediction')
    pred_saved_dir = os.path.join(save_dir, 'pseudo_color_prediction')

    logger.info("Start to predict...")
    progbar_pred = progbar.Progbar(target=len(img_lists[0]), verbose=1)
    with paddle.no_grad():
        for i, im_path in enumerate(img_lists[local_rank]):
            im = cv2.imread(im_path)
            ori_shape = im.shape[:2]
            im, _ = transforms(im)
            im = im[np.newaxis, ...]
            im = paddle.to_tensor(im)

            if aug_pred:
                pred = infer.aug_inference(
                    model,
                    im,
                    ori_shape=ori_shape,
                    transforms=transforms.transforms,
                    scales=scales,
                    flip_horizontal=flip_horizontal,
                    flip_vertical=flip_vertical,
                    is_slide=is_slide,
                    stride=stride,
                    crop_size=crop_size)
            else:
                pred = infer.inference(
                    model,
                    im,
                    ori_shape=ori_shape,
                    transforms=transforms.transforms,
                    is_slide=is_slide,
                    stride=stride,
                    crop_size=crop_size)
            pred = paddle.squeeze(pred)
            pred = pred.numpy().astype('uint8')

            # get the saved name
            if image_dir is not None:
                im_file = im_path.replace(image_dir, '')
            else:
                im_file = os.path.basename(im_path)
            if im_file[0] == '/' or im_file[0] == '\\':
                im_file = im_file[1:]

            # 修改预测后的图片
            pred_saved_path = os.path.join(save_dir, im_file.rsplit(".")[0] + ".png")
            mkdir(pred_saved_path)
            cv2.imwrite(pred_saved_path, pred)

            progbar_pred.update(i + 1)

In [ ]:
# 模型预测
!python /home/aistudio/work/PaddleSeg/predict.py \
       --config /home/aistudio/work/PaddleSeg/configs/quick_start/ocrnet_hrnet_256x256_ce_and_lovasz.yml \
       --model_path /home/aistudio/output_ocrnet/iter_200000/model.pdparams \
       --image_path /home/aistudio/work/img_testA \
       --save_dir /home/aistudio/work/result

# 参考链接
https://zhuanlan.zhihu.com/p/346862877